# 使用Weaviate与生成式OpenAI模块进行生成式搜索本笔记本适用于以下场景：- 您的数据已经存储在Weaviate中- 您希望将Weaviate与生成式OpenAI模块（[generative-openai](https://weaviate.io/developers/weaviate/modules/reader-generator-modules/generative-openai)）一起使用。

## 先决条件本手册仅涵盖生成式搜索示例，但不涵盖配置和数据导入。为了充分利用本手册，请先完成[入门手册](./getting-started-with-weaviate-and-openai.ipynb)，在那里您将学习如何使用Weaviate并导入演示数据。检查清单：* 完成了[入门手册](./getting-started-with-weaviate-and-openai.ipynb)，* 创建了一个`Weaviate`实例，* 将数据导入到您的`Weaviate`实例中，* 拥有[OpenAI API密钥](https://beta.openai.com/account/api-keys)。

===========================================================## 准备您的OpenAI API密钥`OpenAI API密钥` 用于在导入数据时对数据进行向量化，并用于运行查询。如果您还没有OpenAI API密钥，可以从[https://beta.openai.com/account/api-keys](https://beta.openai.com/account/api-keys)获取。获取到密钥后，请将其添加到您的环境变量中，命名为 `OPENAI_API_KEY`。

In [ ]:
# 导出 OpenAI API 密钥!export OPENAI_API_KEY="your key"

In [ ]:
# 确保你的 OpenAI API 密钥已正确设置为环境变量。# 注意：如果你在本地运行这个笔记本，你需要重新加载终端和笔记本，以便环境变量生效。import os# 注意：或者，您也可以像这样设置一个临时的环境变量：# os.environ["OPENAI_API_KEY"] = 'your-key-goes-here'if os.getenv("OPENAI_API_KEY") is not None:    print ("OPENAI_API_KEY is ready")else:    print ("OPENAI_API_KEY environment variable not found")

## 连接到您的Weaviate实例在本节中，我们将：1. 测试环境变量 `OPENAI_API_KEY` - **确保**您已完成[#准备您的OpenAI API密钥](#准备您的OpenAI API密钥)中的步骤2. 使用您的`OpenAI API密钥`连接到您的Weaviate3. 并测试客户端连接### 客户端完成这一步之后，`client`对象将被用于执行所有与Weaviate相关的操作。

In [ ]:
import weaviatefrom datasets import load_datasetimport os# 连接到您的 Weaviate 实例client = weaviate.Client(    url="https://your-wcs-instance-name.weaviate.network/",    # url="http://localhost:8080/",    auth_client_secret=weaviate.auth.AuthApiKey(api_key="<YOUR-WEAVIATE-API-KEY>"), # 如果您没有为您的 Weaviate 实例使用身份验证（例如，对于本地部署的实例），请注释掉这一行。    additional_headers={        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")    })# 检查您的实例是否已上线并准备就绪# 这应该返回 `True`client.is_ready()

## 生成式搜索Weaviate提供了一个[生成式搜索OpenAI](https://weaviate.io/developers/weaviate/modules/reader-generator-modules/generative-openai)模块，该模块根据存储在您的Weaviate实例中的数据生成响应。构建生成式搜索查询的方式与在Weaviate中进行标准语义搜索查询非常相似。例如：* 在“文章”中搜索，* 返回“标题”、“内容”、“URL”* 查找与“足球俱乐部”相关的对象* 将结果限制为5个对象```    result = (        client.query        .get("Articles", ["title", "content", "url"])        .with_near_text("concepts": "football clubs")        .with_limit(5)        # 生成式查询将放在这里        .do()    )```现在，您可以添加`with_generate()`函数来应用生成式转换。`with_generate`接受以下内容之一：- `single_prompt` - 为每个返回的对象生成响应，- `grouped_task` - 从所有返回的对象中生成单个响应。

In [ ]:
def generative_search_per_item(query, collection_name):    prompt = "Summarize in a short tweet the following content: {content}"    result = (        client.query        .get(collection_name, ["title", "content", "url"])        .with_near_text({ "concepts": [query], "distance": 0.7 })        .with_limit(5)        .with_generate(single_prompt=prompt)        .do()    )        # 检查错误    if ("errors" in result):        print ("\033[91mYou probably have run out of OpenAI API calls for the current minute – the limit is set at 60 per minute.")        raise Exception(result["errors"][0]['message'])        return result["data"]["Get"][collection_name]

In [ ]:
query_result = generative_search_per_item("football clubs", "Article")for i, article in enumerate(query_result):    print(f"{i+1}. { article['title']}")    print(article['_additional']['generate']['singleResult']) # 打印生成的响应    print("-----------------------")

In [79]:
def generative_search_group(query, collection_name):    generateTask = "Explain what these have in common"    result = (        client.query        .get(collection_name, ["title", "content", "url"])        .with_near_text({ "concepts": [query], "distance": 0.7 })        .with_generate(grouped_task=generateTask)        .with_limit(5)        .do()    )        # 检查错误    if ("errors" in result):        print ("\033[91mYou probably have run out of OpenAI API calls for the current minute – the limit is set at 60 per minute.")        raise Exception(result["errors"][0]['message'])        return result["data"]["Get"][collection_name]

In [ ]:
query_result = generative_search_group("football clubs", "Article")print (query_result[0]['_additional']['generate']['groupedResult'])

感谢您的跟随，现在您已经具备设置自己的向量数据库并使用嵌入来完成各种酷炫操作的能力 - 祝您使用愉快！对于更复杂的用例，请继续阅读本存储库中的其他示例。